<img src="https://i.ibb.co/hcrKx44/Weekly-Challenge-Banner.png" >

# Weekly Challenge 5
##  Description
Welcome to the 1st challenge! This semester, 6 challenges will be proposed, with increasing difficulty:
* 2 "easy" challenges (challenges 1-2)
* 2 "medium" challenges (challenges 3-4)
* 2 "hard" challenges (challenges 5-6)


## The task
 
After a tiring day of EPFL classes, you decide to treat yourself to FIFA 22 video game for the night. Before you start the game, you have to manage your own team in oder to compete with your homies. In the dataset you have the basic information of the all the current players. 

The dataset _players_data.csv_ is adapted from [FIFA 22 Complete Player Dataset](https://www.kaggle.com/cashncarry/fifa-22-complete-player-dataset) on Kaggle.
Here's a brief description of the dataset:
- FullName
- Overall: overall score of the player
- Positions: players position 
- ValueEUR: the market value of the player in EUR

**Challenge:** Suppose you have just founded a new football club and is ambitious enough to build the best team in the world. If you have a budget of 36 million EUR, what is the most competitive team you can make? Name the players and their position, show the average Overall score of your best team.
*Hints:* 
1. Consider 'ValueEUR' as the purchase price of the player, and try to maximize the total 'Overall' score
2. Suppose you only need 11 players and decide on a 4-3-3 holding (1 GK, 1 LB, 1 RB, 2 CB, 1 CDM, 2 CM, 1 LW, 1 RW, 1 ST)
3. For simplicity, in case of multiple positions, keep the first one. Later you might need to create dummy variables for it.
4. Filter out the players with positions we don't need, and keep only the top 200 (in terms of Overall score) players for each position (if you use the whole dataset, you'll face unbearable running time)
5. You might solve a linear programming system by using package `cvxpy`, for more information please check out the [documentation](https://www.cvxpy.org/)

In [45]:
import numpy as np
import pandas as pd
import cvxpy as cp
import warnings
warnings.filterwarnings("ignore")

In [46]:
df = pd.read_csv("fifa22_players_data.csv")
# keep the first position
df["Positions"] = df["Positions"].str.split(',').str[0]
# filter out positions we don't need
target_positions = ["GK","LB","RB","CB","CDM","CM","LW","RW","ST"]
df = df[df["Positions"].isin(target_positions)] 

In [47]:
# keep only the top 200 (in terms of Overall score) players for each position
reduced_df = df.sort_values(by=["Positions","Overall"],ascending=False) \
               .groupby("Positions") \
               .apply(lambda x: x[:200])[["FullName","Overall","ValueEUR"]] \
               .reset_index(level=0)
reduced_df = pd.get_dummies(reduced_df, columns=["Positions"], prefix='Positions')

In [48]:
reduced_df.head(3)

,FullName,Overall,ValueEUR,Positions_CB,Positions_CDM,Positions_CM,Positions_GK,Positions_LB,Positions_LW,Positions_RB,Positions_RW,Positions_ST
16,Virgil van Dijk,89,86000000,1,0,0,0,0,0,0,0,0
23,Sergio Ramos García,88,24000000,1,0,0,0,0,0,0,0,0
38,Marcos Aoás Corrêa,87,90500000,1,0,0,0,0,0,0,0,0


In [49]:
len(reduced_df)

1800

In [50]:
x = cp.Variable((len(reduced_df),1), boolean=True) # decision variables
c = np.array(reduced_df["Overall"]).reshape(-1,len(reduced_df)) # Overall score of each player
objective = cp.Maximize(c @ x) # objective function: Maximize the Overall score
constraints = [
    np.array(reduced_df["ValueEUR"]).reshape(-1,len(reduced_df)) @ x <= 36000000,
    np.array(reduced_df["Positions_GK"]).reshape(-1,len(reduced_df)) @ x == 1,
    np.array(reduced_df["Positions_LB"]).reshape(-1,len(reduced_df)) @ x == 1,
    np.array(reduced_df["Positions_RB"]).reshape(-1,len(reduced_df)) @ x == 1,
    np.array(reduced_df["Positions_CB"]).reshape(-1,len(reduced_df)) @ x == 2,
    np.array(reduced_df["Positions_CDM"]).reshape(-1,len(reduced_df)) @ x == 1,
    np.array(reduced_df["Positions_CM"]).reshape(-1,len(reduced_df)) @ x == 2,
    np.array(reduced_df["Positions_LW"]).reshape(-1,len(reduced_df)) @ x == 1,
    np.array(reduced_df["Positions_RW"]).reshape(-1,len(reduced_df)) @ x == 1,
    np.array(reduced_df["Positions_ST"]).reshape(-1,len(reduced_df)) @ x == 1
]
problem = cp.Problem(objective, constraints) # problem
problem.solve(solver=cp.GLPK_MI) # solve

857.0

In [51]:
reduced_df[x.value.flatten() > 0]

,FullName,Overall,ValueEUR,Positions_CB,Positions_CDM,Positions_CM,Positions_GK,Positions_LB,Positions_LW,Positions_RB,Positions_RW,Positions_ST
225,Képler Laveran Lima Ferreira,82,5500000,1,0,0,0,0,0,0,0,0
350,José Miguel da Rocha Fonte,81,4600000,1,0,0,0,0,0,0,0,0
153,Fernando Luiz Rosa,83,7000000,0,1,0,0,0,0,0,0,0
1237,José Sosa,76,2400000,0,0,1,0,0,0,0,0,0
1240,Steven Davis,76,2400000,0,0,1,0,0,0,0,0,0
260,Łukasz Fabiański,82,3400000,0,0,0,1,0,0,0,0,0
1955,Enaldo Diogo Barbosa Prazeres,75,1200000,0,0,0,0,1,0,0,0,0
935,Mathieu Valbuena,77,3400000,0,0,0,0,0,1,0,0,0
1460,Mathieu Debuchy,75,1700000,0,0,0,0,0,0,1,0,0
1824,José Pedro Fuenzalida,75,1800000,0,0,0,0,0,0,0,1,0


In [52]:
print("The highest average Overall score of the team is %d" % (857/11))

The highest average Overall score of the team is 77


Best team:
- CB: Képler Laveran Lima Ferreira & José Miguel da Rocha Fonte
- CDM: Fernando Luiz Rosa
- CM: José Sosa & Steven Davis
- GK: Łukasz Fabiański
- LB: Enaldo Diogo Barbosa Prazeres
- LW: Mathieu Valbuena
- RB: Mathieu Debuchy
- RW: José Pedro Fuenzalida
- ST: Mauro Boselli
